In [96]:
from sklearn.datasets import fetch_olivetti_faces
#人脸数据集，由40个人脸组成，每人10张照片组成

In [ ]:
#1、导入包
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor,Lambda,Compose
import matplotlib.pyplot as plt

In [225]:
#2、下载数据并打包
olivetti_faces = fetch_olivetti_faces(data_home='./face_data', shuffle=True)
print(olivetti_faces.data.shape)
print(olivetti_faces.target.shape)
print(olivetti_faces.images.shape)
batch_size = 50
olivetti_faces.images = torch.tensor(olivetti_faces.images,dtype=torch.float)
olivetti_faces.target = torch.tensor(olivetti_faces.target, dtype=torch.long)     

train_datasets = list(zip(olivetti_faces.images[:350,:,:],olivetti_faces.target[:350])) #64*64,target 30
test_datasets = list(zip(olivetti_faces.images[350:,:,:],olivetti_faces.target[350:])) #64*64,target 30

train_data = DataLoader(train_datasets,batch_size=batch_size,shuffle=True)
test_data = DataLoader(test_datasets,batch_size=batch_size)

(400, 4096)
(400,)
(400, 64, 64)


In [220]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [221]:
#2、定义模型 model_1
class facedet(nn.Module):
    def __init__(self):
        super().__init__()
        self.det_ppl = nn.Sequential(
            nn.Linear(4096,4096),
            nn.BatchNorm1d(4096),
            nn.LeakyReLU(),
            nn.Linear(4096,2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(),
            nn.Linear(2048,1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(1024,512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(128,40),
        )
    def forward(self, x):
        x = self.det_ppl(x)
        return x
model = facedet().to(device)    #把模型网络结构结构放在cpu
print(model)

facedet(
  (det_ppl): Sequential(
    (0): Linear(in_features=4096, out_features=4096, bias=True)
    (1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=4096, out_features=2048, bias=True)
    (4): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=2048, out_features=1024, bias=True)
    (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Linear(in_features=1024, out_features=512, bias=True)
    (10): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Linear(in_features=512, out_features=256, bias=True)
    (13): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): LeakyReLU(negative_slo

In [202]:
#2、定义模型 model_2
class facedet(nn.Module):
    def __init__(self):
        super().__init__()
        self.det_ppl = nn.Sequential(
            nn.Linear(4096,4096),
            nn.Dropout(p=0.3),
            nn.BatchNorm1d(4096),
            nn.LeakyReLU(),
            nn.Linear(4096,2048),
            nn.BatchNorm1d(2048),
            nn.LeakyReLU(),
            nn.Linear(2048,1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            # nn.Dropout(p=0.3),
            nn.Linear(1024,512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(128,40),
        )
    def forward(self, x):
        x = self.det_ppl(x)
        return x
model = facedet().to(device)    #把模型网络结构结构放在cpu

In [226]:
#2、定义模型 model_3
class facedet(nn.Module):
    def __init__(self):
        super().__init__()
        self.det_ppl = nn.Sequential(
            nn.Linear(4096,4096),
            nn.LeakyReLU(),
            nn.Linear(4096,2048),
            nn.LeakyReLU(),
            nn.Linear(2048,1024),
            nn.LeakyReLU(),
            nn.Linear(1024,512),
            nn.LeakyReLU(),
            nn.Linear(512,256),
            nn.LeakyReLU(),
            nn.Linear(256,128),
            nn.LeakyReLU(),
            nn.Linear(128,40),
        )
    def forward(self, x):
        x = self.det_ppl(x)
        return x
model = facedet().to(device)    #把模型网络结构结构放在cpu

In [227]:
#3、定义优化器和损失函数
LR = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=LR)
# optimier = torch.optim.SGD(model.parameters(), LR)

In [228]:
#4、训练函数
loss_list,acc_list = [],[]
def train_func(data,loss_fn,optimizer,model):
    model.train()
    for batch,(da,target) in enumerate(data):
        da,target = da.to(device),target.to(device)
        da = da.reshape(50,-1)
        pred = model(da)
        loss = loss_fn(pred,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch%50==0:
            print(f'train batch:{batch},loss:{loss}')
def test_func(model,data,loss_fn):
    model.eval()
    total,cor,it,cur =0,0,0,0
    loss_test = 0
    with torch.no_grad():
        for bs,(x,y) in enumerate(data):
            it += 1
            total += y.size(0)
            x = x.reshape(50,-1)
            pred = model(x)
            _,predtied = torch.max(pred,1)
            cor += (predtied == y).sum().item()
            cur = loss_fn(pred,y)
            loss_list.append(cur)
            loss_test += (cur).sum().item()
        print(f'Test mean loss:{(loss_test/it):.3f}')
        acc = cor/total
        acc_list.append(acc)
        print(f'Test Acc:{acc:.3f}') 

In [229]:
#6、训练
epochs = 10
for i in range(epochs):
    train_func(train_data,loss_fn,optimizer,model)
    test_func(model,test_data,loss_fn)
    print('_____________________________________')
print(f'结束训练')

train batch:0,loss:3.6915993690490723
Test mean loss:3.738
Test Acc:0.040
_____________________________________
train batch:0,loss:3.6659913063049316
Test mean loss:3.708
Test Acc:0.000
_____________________________________
train batch:0,loss:3.6636838912963867
Test mean loss:3.699
Test Acc:0.020
_____________________________________
train batch:0,loss:3.6827712059020996
Test mean loss:3.724
Test Acc:0.000
_____________________________________
train batch:0,loss:3.6820013523101807
Test mean loss:3.768
Test Acc:0.000
_____________________________________
train batch:0,loss:3.7132928371429443
Test mean loss:3.779
Test Acc:0.000
_____________________________________
train batch:0,loss:3.6790826320648193
Test mean loss:3.747
Test Acc:0.000
_____________________________________
train batch:0,loss:3.6869161128997803
Test mean loss:3.778
Test Acc:0.000
_____________________________________
train batch:0,loss:3.6834466457366943
Test mean loss:3.869
Test Acc:0.000
______________________________

#### model_1 效果最好
    train batch:0,loss:0.25418585538864136
    Test mean loss:0.768
    Test Acc:0.920
#### model_2 dropout效果差
    train batch:0,loss:0.9830032587051392
    Test mean loss:4.343
    Test Acc:0.020
    添加dropout后准确率那么低吗？待商榷
#### model_1换成SGD优化器效果差
    train batch:0,loss:3.8363780975341797
    Test mean loss:3.669
    Test Acc:0.080
#### model_3 紧使用线性和激活梯度消失？
    train batch:0,loss:3.6177749633789062
    Test mean loss:3.706
    Test Acc:0.000